In [1]:
import os
if "models" not in os.listdir("."):
    os.chdir("../..")

In [2]:
%load_ext autoreload
%autoreload 2
import penzai
import jax_smi
jax_smi.initialise_tracking()
from penzai import pz
pz.ts.register_as_default()
pz.ts.register_autovisualize_magic()
pz.enable_interactive_context()

In [3]:
from micrlhf.llama import LlamaTransformer
llama = LlamaTransformer.from_pretrained("models/gemma-2b-it.gguf", from_type="gemma", load_eager=True, device_map="tpu:0")

In [4]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("alpindale/gemma-2b")
tokenizer.padding_side = "right"

In [5]:
from sprint.task_vector_utils import load_tasks, ICLRunner
tasks = load_tasks()

fatal: destination path 'data/itv' already exists and is not an empty directory.


In [6]:
# task_names = ["en_es", "antonyms", "person_profession", "es_en", "present_simple_gerund", "present_simple_past_simple", "person_profession", "person_language", "country_capital", "football_player_position"]
# task_name = task_names[1]
task_names = list(tasks.keys())

In [7]:
import jax.numpy as jnp
import jax


from sprint.task_vector_utils import ICLRunner, logprob_loss, get_tv, make_act_adder
from micrlhf.llama import LlamaBlock
from micrlhf.sampling import sample, jit_wrapper


get_resids = llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(lambda i, x:
    pz.nn.Sequential([
        pz.de.TellIntermediate.from_config(tag=f"resid_pre_{i}"),
        x
    ])
)
get_resids = pz.de.CollectingSideOutputs.handling(get_resids, tag_predicate=lambda x: x.startswith("resid_pre"))
get_resids_call = jit_wrapper.Jitted(get_resids)



def tokenized_to_inputs(input_ids, attention_mask):
    token_array = jnp.asarray(input_ids)
    token_array = jax.device_put(token_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    token_array = pz.nx.wrap(token_array, "batch", "seq").untag("batch").tag("batch")

    mask_array = jnp.asarray(attention_mask, dtype=jnp.bool)
    mask_array = jax.device_put(mask_array, jax.sharding.NamedSharding(llama.mesh, jax.sharding.PartitionSpec("dp", "sp")))
    mask_array = pz.nx.wrap(mask_array, "batch", "seq").untag("batch").tag("batch")

    inputs = llama.inputs.from_basic_segments(token_array)
    return inputs


In [8]:
from sprint.icl_sfc_utils import AblatedModule
layer = 12
mask_name = "arrow"

In [9]:
from micrlhf.utils.load_sae import get_nev_it_sae_suite, sae_encode


sae = get_nev_it_sae_suite(layer=layer)

In [10]:
import json

with open("cleanup_results_final.jsonl") as f:
    lines = f.readlines()
    results = [json.loads(line) for line in lines]

with open("cleanup_results_algo.jsonl") as f:
    lines = f.readlines()
    results_algo = [json.loads(line) for line in lines]

In [12]:
import numpy as np

features = []
task_features = {}

for task_name in task_names:
    if not task_name.startswith("algo"):
        task_results = [result for result in results if result["task"] == task_name and result["layer"] == layer]
    else:
        task_results = [result for result in results_algo if result["task"] == task_name and result["layer"] == layer]

    for result in task_results:
        weights = np.array(result["weights"])
        # s = jax.nn.softplus(sae["s_gate"]) * sae["scaling_factor"]
        # threshold = jnp.maximum(0, sae["b_gate"] - sae["b_enc"] * s)
        # w = weights
        # w = w * (w > 0)

        _, w, _ = sae_encode(sae, None, pre_relu=weights)

        # print(threshold)

        new_features = np.nonzero(w)[0].tolist()
        features += new_features
        print(task_name, "TVC:", result["loss"], "TV:", result["tv_loss"], new_features)

        task_features[task_name] = new_features
    

features = list(set(features))

len(features)

location_continent TVC: 7.28125 TV: 5.375 [6780, 7578, 9662, 14612, 24925]
football_player_position TVC: 11.0625 TV: 9.125 [7491, 13458, 27401]
location_religion TVC: 4.96875 TV: 6.75 [7578, 9600, 9662, 11172, 14612, 20832, 24925, 27401]
location_language TVC: 5.375 TV: 5.28125 [7578, 9662, 11172, 24925]
person_profession TVC: 6.5 TV: 6.03125 [6413, 7491, 7578, 13458, 27401]
location_country TVC: 3.625 TV: 4.25 [850, 9662, 11173, 24925]
country_capital TVC: 2.171875 TV: 2.046875 [11173, 17636, 18803]
person_language TVC: 4.5625 TV: 4.25 [850, 7578, 11172, 13458, 24925]
singular_plural TVC: 4.8125 TV: 3.59375 [2930, 6594, 12943, 14612]
present_simple_past_simple TVC: 1.5703125 TV: 0.53515625 [2930, 6594, 15356]
antonyms TVC: 2.234375 TV: 2.40625 [7578, 7739, 10720, 11618, 19097, 19112, 25576]
plural_singular TVC: 3.078125 TV: 3.0625 [2930, 6594]
present_simple_past_perfect TVC: 3.328125 TV: 3.046875 [2930, 6594, 15356]
present_simple_gerund TVC: 3.25 TV: 2.125 [6594, 15554]
en_it TVC: 9

43

In [13]:
tvs = {}

for task_name in task_names:
    if not task_name.startswith("algo"):
        task_result = [result for result in results if result["task"] == task_name and result["layer"] == layer][0]
    else:
        task_result = [result for result in results_algo if result["task"] == task_name and result["layer"] == layer][0]

    tvs[task_name] = np.array(task_result["tv"])

In [19]:
import dataclasses
from tqdm.auto import tqdm
from functools import partial
from micrlhf.utils.activation_manipulation import add_vector

task_losses_positive = {}

n_few_shots, batch_size, max_seq_len = 20, 16, 256
seed = 10

prompt = "Follow the pattern:\n{}"

def make_taker(llama, layer):
    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    return taker

taker = make_taker(llama, layer)

for task_name in tqdm(task_names):

    sep = 3978
    pad = 0


    pairs = list(tasks[task_name].items())

    n_shot = n_few_shots - 1
    if task_name.startswith("algo"):
        n_shot = 8

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=1, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

    tokenized = runner.get_tokens([
        x[:n_few_shots] for x in runner.train_pairs
    ], tokenizer)

    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, all_resids = get_resids_call(inputs)

    scale = 30

    resids = all_resids[layer].value.unwrap("batch", "seq", "embedding")

    mask = train_tokens == sep
    col_indices = jnp.arange(mask.shape[1])
    col_indices_broadcasted = mask * col_indices
    sorted_indices = jnp.sort(col_indices_broadcasted, axis=1, descending=True)
    positions = sorted_indices[:, :1]

    tv = tvs[task_name]
    
    def steer_with_direction(direction, scale):
        direction = direction / jnp.linalg.norm(direction)
        direction = direction * scale
        direction = tv - direction
        
        modified = jax.vmap(lambda a, b: a.at[b].add(direction))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        _inputs = dataclasses.replace(inputs, tokens=modified)
        logits = taker(_inputs).unwrap("batch", "seq", "vocabulary")

        return logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2, do_ppl=False)

    task_losses_positive[task_name] = [[steer_with_direction(sae["W_dec"][feature], scale).tolist() for feature in tqdm(features)]]

    task_losses_positive[task_name].append(steer_with_direction(1, 0).tolist())

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning:

scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.



  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

...

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

In [21]:
for task_name in task_names:
    print(task_name, task_losses_positive[task_name][1])
    # 1/0

location_continent 5.03125
football_player_position 8.75
location_religion 6.59375
location_language 5.65625
person_profession 6.0
location_country 4.53125
country_capital 2.5
person_language 4.40625
singular_plural 4.625
present_simple_past_simple 0.85546875
antonyms 2.71875
plural_singular 2.125
present_simple_past_perfect 2.671875
present_simple_gerund 2.25
en_it 13.25
it_en 6.53125
en_fr 14.6875
en_es 8.875
fr_en 5.40625
es_en 2.578125
algo_last 0.62890625
algo_first 0.423828125
algo_second 4.9375


In [27]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    # losses = np.minimum(losses, base_loss)

    losses = (losses - base_loss) / base_loss
    
    # max_loss = np.max(losses)
    # min_loss = np.min(losses)

    # losses = (losses - min_loss) / (max_loss - min_loss)

    # losses = np.clip(losses, 0.2, 10)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

def make_heatmap():
    normalized_losses = {}

    drop_features = []
    drop_ids = [features.index(feature) for feature in drop_features]

    features_dropped = [feature for feature in features if feature not in drop_features]

    for task_name, losses in task_losses_positive.items():
        base_loss = losses[1]
        losses = losses[0]
        losses = np.array(losses)
        losses = np.delete(losses, drop_ids)

        losses = np.minimum(losses, base_loss)

        losses = (base_loss - losses) / base_loss
        
        max_loss = np.max(losses)
        min_loss = np.min(losses)

        losses = (losses - min_loss) / (max_loss - min_loss)

        losses = np.clip(losses, 0.2, 10)
        
        # mean_loss = np.mean(losses - base_loss)

        # base_acc = losses[1][1]

        # accs = [base_acc - loss[1] for loss in losses[0]]
        normalized_losses[task_name] = losses


    heatmap = np.zeros((len(task_names), len(features_dropped)))

    for i, task_name in enumerate(task_names):
        for j, feature in enumerate(features_dropped):
            heatmap[i, j] = normalized_losses[task_name][j]

    return heatmap


# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.log(1 + heatmap * 40)

# heatmap = np.clip(heatmap, -5, 5)

# avg_heatmap = np.sum(heatmap != 1, axis=0)

sort_map = make_heatmap()
sort_map = heatmap

avg_heatmap  = np.max(sort_map, axis=0)

sorted_idx = np.argsort(-avg_heatmap)

heatmap = heatmap[:, sorted_idx]

sort_map = sort_map[:, sorted_idx]

# min_heatmap = np.min(heatmap, axis=1)   

min_pos = np.argmax(sort_map, axis=1)

y_sorted_idx = np.argsort(min_pos)



heatmap = heatmap[y_sorted_idx]

fig = px.imshow(heatmap, x=[str(features_dropped[x]) for x in sorted_idx], y=[task_names[x] for x in y_sorted_idx], width=1000, height=600, aspect="auto", color_continuous_scale="Blues")

def make_filp_mask():
    heatmap = np.zeros((len(task_names), len(features)))
    feat_to_idx = {feat: i for i, feat in enumerate(features)}


    for i, task_name in enumerate(task_names):
        feats = task_features[task_name]
        for feat in feats:
            if feat in feat_to_idx:
                heatmap[i, feat_to_idx[feat]] = -2

    heatmap = heatmap[y_sorted_idx]
    heatmap = heatmap[:, sorted_idx]

    return heatmap + 1



filp_mask = make_filp_mask()

annotation_style = dict(
    showarrow=False,
    font=dict(color="white", size=5),
    xanchor="center",
    yanchor="middle"
)

special_cells = {
    tuple(x): ["🔴"] for x in np.argwhere(filp_mask == -1)
}

annotations = [
    {**annotation_style, **dict(x=cell[1], y=cell[0], text=icon, yshift=i * 10)}
    for cell, icons in special_cells.items()
    for i, icon in enumerate(icons)
]
fig.update_layout(annotations=annotations)

fig.show()


/tmp/ipykernel_896928/1224513659.py:64: RuntimeWarning:

invalid value encountered in divide



In [24]:
import dataclasses
from tqdm.auto import tqdm
from functools import partial
from micrlhf.utils.activation_manipulation import add_vector, add_vector_many

positive_task_losses_testing = {}

n_few_shots, batch_size, max_seq_len = 1, 16, 64
seed = 10

prompt = "Follow the pattern:\n{}"


def calc_acc(tokens, sep, logits):
    preds = logits.argmax(-1)
    preds = preds[:, :-1]
    targets = tokens[:, 1:]

    mask = tokens[:, :-1] == sep

    hits = targets == preds

    hits = hits * mask

    hits = hits.sum()
    return hits / mask.sum()




def make_taker(llama, layer):
    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    return taker

taker = make_taker(llama, layer)

for task_name in tqdm(["antonyms"]):

    sep = 3978
    pad = 0
    pairs = list(tasks[task_name].items())

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=1, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

    tokenized = runner.get_tokens([
        x[:n_few_shots] for x in runner.train_pairs
    ], tokenizer)

    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, all_resids = get_resids_call(inputs)

    resids = all_resids[layer].value.unwrap("batch", "seq", "embedding")

    mask = train_tokens == sep
    col_indices = jnp.arange(mask.shape[1])
    col_indices_broadcasted = mask * col_indices
    sorted_indices = jnp.sort(col_indices_broadcasted, axis=1, descending=True)
    positions = sorted_indices[:, :1]

    tv = tvs[task_name]

    def steer_with_direction(direction, scale):
        direction = direction / jnp.linalg.norm(direction)
        direction = direction * scale
        direction = tv - direction
        
        modified = jax.vmap(lambda a, b: a.at[b].add(direction))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        _inputs = dataclasses.replace(inputs, tokens=modified)

        logits = taker(_inputs).unwrap("batch", "seq", "vocabulary")

        acc = calc_acc(train_tokens, sep, logits)

        return logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2), acc

    # _features = task_features[task_name]

    logits = llama(inputs)

    logits = logits.unwrap("batch", "seq", "vocabulary")

    acc = calc_acc(train_tokens, sep, logits)


    for feature in tqdm(features):
        positive_task_losses_testing[(task_name, feature)] = [[steer_with_direction(sae["W_dec"][feature], scale) for scale in np.logspace(0, 1.5, 50)]]
        positive_task_losses_testing[(task_name, feature)].append((logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2), acc))

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/43 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning:

scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning:

scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.



In [29]:


feature = 11618

heatmap = np.zeros((len(task_names), 50))

for i, task_name in enumerate(task_names):
    losses, base_loss = positive_task_losses_testing[(task_name, feature)]
    # print(task_name, base_loss[0])
    if task_name.startswith("algo"):
        continue
    # if base_loss[1] < 0.4:
    #     continue
    # negative_task_losses_testing[(task_name, feature)]
    heatmap[i] =  float(base_loss[0]) - np.array([float(loss) for loss, _  in losses])
    heatmap[i] = heatmap[i] / float(base_loss[0])
    # heatmap[i] = heatmap[i] / float(base_loss[1])
# clipped = np.maximum(heatmap, 0)
# heatmap /= clipped.mean(axis=0)
# print(clipped.mean(0).shape)

px.imshow(heatmap, x=np.logspace(0, 1.5, 50), y=task_names, height=600)

KeyError: ('location_continent', 11618)

In [30]:


# feature = 11618

task_name = "antonyms"

heatmap = np.zeros((len(features), 50))

for i, feature in enumerate(features):
    losses, base_loss = positive_task_losses_testing[(task_name, feature)]
    # print(task_name, base_loss[0])
    # if task_name.startswith("algo"):
    #     continue
    # if base_loss[1] < 0.4:
    #     continue
    # negative_task_losses_testing[(task_name, feature)]
    heatmap[i] = np.array([float(loss) for loss, _  in losses]) - float(base_loss[0])
    heatmap[i] = heatmap[i] / float(base_loss[0])
    # heatmap[i] = heatmap[i] / float(base_loss[1])
# clipped = np.maximum(heatmap, 0)
# heatmap /= clipped.mean(axis=0)
# print(clipped.mean(0).shape)

px.imshow(heatmap, x=np.logspace(0, 1.5, 50), y=[str(x) for x in features], height=600)

In [112]:
import plotly.express as px

heatmap = np.zeros((len(task_names), len(features)))    

for i, feature in enumerate(features):
    run_data = np.zeros((len(task_names), 50))
    for j, task_name in enumerate(task_names):
        losses, base_loss = positive_task_losses_testing[(task_name, feature)]
        n_loss = 0
        # if task_name.startswith("algo"):
        #     continue
        # if base_loss[1] < 0.3:
        #     continue
        # if task_name == "present_simple_gerund":
        #     continue
        run_data[j] = float(base_loss[n_loss]) - np.array([float(loss[n_loss]) for loss  in losses])
        # print(
        #     task_name, base_loss
        # )
        run_data[j] = run_data[j] / float(base_loss[n_loss])
        # run_data[j] *= -1
    # run_data = run_data.max(0)
    # find first idx where > 1
    first_idx = np.argmax(run_data.max(0))

    # print(first_idx)
    
    # run_data = np.maximum(run_data, 0)
    run_data[run_data < 0] = 0
    # run_data[run_data > 0.1] = 0.3
    heatmap[:, i] = run_data[:, first_idx]
    # heatmap[:, i] = run_data[:, first_idx]
    # heatmap[:, i] = heatmap[:, i] / heatmap[:, i].mean() 

# heatmap = heatmap / heatmap.mean(0)


# heatmap = np.maximum(heatmap, 0)
sorted_indices = np.argsort((heatmap).max(0))

sorted_indices = sorted_indices[::-1]

heatmap = heatmap[:, sorted_indices]

min_pos = np.argmax(heatmap, axis=1)

y_sorted_idx = np.argsort(min_pos)

heatmap = heatmap[y_sorted_idx]

def make_filp_mask():
    heatmap = np.zeros((len(task_names), len(features)))
    feat_to_idx = {feat: i for i, feat in enumerate(features)}


    for i, task_name in enumerate(task_names):
        feats = task_features[task_name]
        for feat in feats:
            if feat in feat_to_idx:
                heatmap[i, feat_to_idx[feat]] = -2

    heatmap = heatmap[y_sorted_idx]
    heatmap = heatmap[:, sorted_indices]

    return heatmap + 1



filp_mask = make_filp_mask()
fig = px.imshow(heatmap, x=[str(features[i]) for i in sorted_indices] ,  y=[task_names[i] for i in y_sorted_idx] , height=600, color_continuous_scale="Blues")

special_cells = {
    tuple(x): ["🔴"] for x in np.argwhere(filp_mask == -1)
}

for cell, icons in special_cells.items():
    for i, icon in enumerate(icons):
        fig.add_annotation(
            x=cell[1],  # Column index
            y=cell[0],  # Row index
            text=icon,  # Icon or text
            showarrow=False,
            font=dict(color="white", size=5),
            xanchor="center",
            yanchor="middle",
            yshift=i * 10  # Adjust vertical position to stack icons
        )

fig


In [19]:
with open("micrlhf-progress/executor_heatmap_l12.json", "w") as f:
    json.dump({"heatmap": heatmap.tolist(), "features": [features_dropped[x] for x in sorted_idx], "task_names": [task_names[x] for x in y_sorted_idx]}, f)

In [24]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = np.minimum(losses, base_loss)

    losses = (losses - base_loss) / base_loss
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / (max_loss - min_loss)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)

# avg_heatmap = np.sum(heatmap != 1, axis=0)

avg_heatmap  = np.mean(heatmap, axis=0)

sorted_idx = np.argsort(avg_heatmap)

heatmap = heatmap[:, sorted_idx]


fig = px.imshow(heatmap, x=[str(features_dropped[x]) for x in sorted_idx], y=task_names, width=1400, height=400, aspect="auto")

fig.show()


In [34]:
tvs = [x["tv"] for x in results if x["layer"] == layer]

len(tvs)

23

In [43]:
avg_tv = np.mean(tvs, axis=0)

avg_tv = avg_tv.astype(jnp.float32)

In [58]:
import dataclasses
from tqdm.auto import tqdm
from functools import partial
from micrlhf.utils.activation_manipulation import add_vector

task_losses_positive = {}

n_few_shots, batch_size, max_seq_len = 20, 16, 256
seed = 10

prompt = "Follow the pattern:\n{}"

def make_taker(llama, layer):
    taker = jit_wrapper.Jitted(llama.select().at_instances_of(LlamaBlock).apply_with_selected_index(
        lambda i, x: x if i >= layer else pz.nn.Identity()
    ).select().at_instances_of(pz.nn.EmbeddingLookup).apply(lambda _: pz.nn.Identity())
                    .select().at_instances_of(pz.nn.ConstantRescale).pick_nth_selected(0).apply(lambda _: pz.nn.Identity()))

    return taker

taker = make_taker(llama, layer)

for task_name in tqdm(["location_religion"]):

    sep = 3978
    pad = 0


    pairs = list(tasks[task_name].items())

    n_shot = n_few_shots - 1
    if task_name.startswith("algo"):
        n_shot = 8

    runner = ICLRunner(task_name, pairs, batch_size=batch_size, n_shot=1, max_seq_len=max_seq_len, seed=seed, prompt=prompt)

    tokenized = runner.get_tokens([
        x[:n_few_shots] for x in runner.train_pairs
    ], tokenizer)

    inputs = tokenized_to_inputs(**tokenized)
    train_tokens = tokenized["input_ids"]

    _, all_resids = get_resids_call(inputs)

    # scale = 25

    resids = all_resids[layer].value.unwrap("batch", "seq", "embedding")

    mask = train_tokens == sep
    col_indices = jnp.arange(mask.shape[1])
    col_indices_broadcasted = mask * col_indices
    sorted_indices = jnp.sort(col_indices_broadcasted, axis=1, descending=True)
    positions = sorted_indices[:, :1]
    
    feature = 11172

    def steer_with_direction(direction, scale):
        direction = direction / jnp.linalg.norm(direction)
        direction = direction * scale + avg_tv * 0.5
        
        modified = jax.vmap(lambda a, b: a.at[b].add(direction))(
            resids, positions
        )
        modified = pz.nx.wrap(modified, "batch", "seq", "embedding")

        _inputs = dataclasses.replace(inputs, tokens=modified)
        logits = taker(_inputs).unwrap("batch", "seq", "vocabulary")

        return logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2)

    task_losses_positive[task_name] = [[steer_with_direction(sae["W_dec"][feature], scale).tolist() for scale in tqdm(np.logspace(0, 2, 100))]]

    logits = llama(inputs)

    logits = logits.unwrap("batch", "seq", "vocabulary")

    task_losses_positive[task_name].append(logprob_loss(logits, train_tokens, sep=sep, pad_token=pad, n_first=2).tolist())

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

/home/dmitrii/.cache/pypoetry/virtualenvs/micrlhf-progress-_SD4q1c9-py3.10/lib/python3.10/site-packages/jax/_src/ops/scatter.py:92: FutureWarning:

scatter inputs have incompatible types: cannot safely cast value from dtype=float32 to dtype=bfloat16 with jax_numpy_dtype_promotion='standard'. In future JAX releases this will result in an error.



In [59]:
px.line(x=np.logspace(0, 2, 100), y=task_losses_positive["location_religion"][0])

In [54]:
px.line(x=np.logspace(0, 2, 100), y=task_losses_positive["antonyms"][0])

In [47]:
task_losses_positive["antonyms"][0]

[6.28125,
 6.28125,
 6.25,
 6.25,
 6.25,
 6.25,
 6.21875,
 6.25,
 6.21875,
 6.21875,
 6.21875,
 6.1875,
 6.1875,
 6.1875,
 6.15625,
 6.125,
 6.125,
 6.125,
 6.125,
 6.09375,
 6.09375,
 6.0625,
 6.03125,
 6.0625,
 6.03125,
 6.0,
 5.96875,
 5.96875,
 5.9375,
 5.9375,
 5.90625,
 5.875,
 5.84375,
 5.8125,
 5.8125,
 5.78125,
 5.71875,
 5.65625,
 5.625,
 5.59375,
 5.5625,
 5.5,
 5.5,
 5.4375,
 5.40625,
 5.34375,
 5.28125,
 5.25,
 5.15625,
 5.09375,
 5.0625,
 4.96875,
 4.90625,
 4.8125,
 4.75,
 4.6875,
 4.59375,
 4.5,
 4.40625,
 4.3125,
 4.21875,
 4.09375,
 4.0,
 3.90625,
 3.8125,
 3.703125,
 3.671875,
 3.609375,
 3.578125,
 3.578125,
 3.640625,
 3.703125,
 3.796875,
 3.90625,
 4.03125,
 4.15625,
 4.3125,
 4.53125,
 4.6875,
 4.875,
 5.09375,
 5.28125,
 5.5625,
 5.8125,
 6.09375,
 6.375,
 6.625,
 6.90625,
 7.1875,
 7.5,
 7.75,
 8.0,
 8.3125,
 8.5625,
 8.8125,
 9.0,
 9.3125,
 9.5625,
 9.8125,
 10.0625]

In [46]:
heatmap.shape

(23, 129)

In [45]:
sorted_idx

array([[ 72, 119, 117, ...,  35,  67, 128],
       [ 13,  33,  69, ...,  44,  56, 128],
       [103, 118,  78, ...,  44,  42,  64],
       ...,
       [106,  27,  15, ...,  42,  37, 128],
       [ 89, 116,  73, ...,  34, 127, 128],
       [ 73,  39,   1, ...,  37,  68, 128]])

In [67]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in negative_task_losses.items():
    base_loss = losses[1]
    losses = [x[1].tolist() for x in losses[0]]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = (losses - base_loss[1]) 
    
    # max_loss = np.max(losses)
    # min_loss = np.min(losses)

    # losses = (losses - min_loss) / (max_loss - min_loss)

    # losses = (losses - min_loss) / (max_loss - min_loss)
    # loss = (base_loss - losses) / base_loss
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)

std_heatmap = np.std(heatmap, axis=0)

sorted_idx = np.argsort(-std_heatmap)

heatmap = heatmap[:, sorted_idx]

labels = [str(features_dropped[x]) for x in sorted_idx]

fig = px.imshow(heatmap, x=labels, y=task_names, width=1400, height=800, aspect="auto")

fig.show()


In [66]:
task_features

{'location_continent': [6780, 7578, 9662, 14612, 24925],
 'football_player_position': [7491, 13458, 27401],
 'location_religion': [7578, 9600, 9662, 11172, 14612, 20832, 24925, 27401],
 'location_language': [7578, 9662, 11172, 24925],
 'person_profession': [6413, 7491, 7578, 13458, 27401],
 'location_country': [850, 9662, 11173, 24925],
 'country_capital': [11173, 17636, 18803],
 'person_language': [850, 7578, 11172, 13458, 24925],
 'singular_plural': [2930, 6594, 12943, 14612],
 'present_simple_past_simple': [2930, 6594, 15356],
 'antonyms': [7578, 7739, 10720, 11618, 19097, 19112, 25576],
 'plural_singular': [2930, 6594],
 'present_simple_past_perfect': [2930, 6594, 15356],
 'present_simple_gerund': [6594, 15554],
 'en_it': [2539, 5991, 6594, 26594],
 'it_en': [5579, 12943, 16490],
 'en_fr': [2539, 2930, 5991],
 'en_es': [5991, 6594, 26594, 26987],
 'fr_en': [5579, 12943, 16490],
 'es_en': [5579, 12943, 16490],
 'algo_last': [2930, 5991, 6594, 12943, 14612, 27268],
 'algo_first': [],
 'algo_second': [6594, 16996, 27268]}

In [120]:
features_dropped.index(7491)

69

In [121]:
mean_heatmap = np.mean(heatmap, axis=0)

drop_features = np.where(mean_heatmap < 0.9)[0]

drop_features = [features_dropped[x] for x in drop_features]

drop_features

[26127,
 6171,
 10787,
 30247,
 3113,
 9782,
 32320,
 15947,
 23682,
 12943,
 16016,
 13458,
 30888,
 19112,
 11491,
 23312,
 7967,
 7491,
 1878,
 20832,
 6503,
 26987,
 2939,
 31620,
 11172,
 25534,
 6594,
 5579,
 23514,
 6629,
 18416]

In [115]:
heatmap.shape

(23, 129)

In [116]:
(22 + 0.1)/23

0.9608695652173914

In [127]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in task_losses_positive.items():
    base_loss = losses[1]
    losses = losses[0]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)


    losses = np.minimum(losses, base_loss * 1.5)

    losses = (losses - base_loss) / base_loss
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / ((max_loss - min_loss) + 1e-6)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

# heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)

heatmap = np.log(heatmap + 1)


fig = px.imshow(heatmap, x=[str(x) for x in features_dropped], y=task_names, color_continuous_scale='YlGnBu')

fig.show()

In [125]:
task_losses_positive["present_simple_past_perfect"]

[[10.3125,
  4.40625,
  4.75,
  6.6875,
  6.0625,
  5.15625,
  5.96875,
  5.9375,
  5.15625,
  7.8125,
  6.84375,
  4.75,
  7.125,
  7.625,
  6.25,
  4.875,
  6.5,
  5.1875,
  4.25,
  4.875,
  5.75,
  9.625,
  6.03125,
  10.5,
  6.6875,
  10.625,
  5.46875,
  9.1875,
  9.625,
  11.125,
  13.125,
  6.90625,
  4.21875,
  5.71875,
  4.75,
  4.71875,
  6.0625,
  5.625,
  5.03125,
  6.6875,
  4.96875,
  6.65625,
  11.1875,
  9.8125,
  7.375,
  4.875,
  10.125,
  6.75,
  7.96875,
  6.8125,
  5.625,
  14.75,
  13.1875,
  7.9375,
  5.28125,
  13.3125,
  6.3125,
  6.71875,
  5.1875,
  19.375,
  6.5,
  6.65625,
  5.25,
  4.6875,
  6.03125,
  7.9375,
  11.625,
  10.4375,
  6.5,
  6.09375,
  6.9375,
  10.0,
  17.0,
  5.15625,
  13.625,
  6.59375,
  5.1875,
  7.1875,
  5.34375,
  13.0,
  4.96875,
  7.3125,
  4.75,
  8.75,
  5.09375,
  5.0,
  17.875,
  6.09375,
  7.1875,
  5.34375,
  6.9375,
  8.5625,
  12.8125,
  5.0625,
  12.75,
  10.6875,
  9.0625,
  10.5,
  5.625,
  11.25,
  6.09375,
  7.1875,
  9.25,
  16.375,
  9.5625,
  5.78125,
  7.125,
  8.3125,
  10.0625,
  10.5625,
  5.125,
  3.53125,
  8.1875,
  6.28125,
  6.03125,
  11.875,
  6.59375,
  4.84375,
  8.875,
  6.1875,
  5.84375,
  5.03125,
  6.78125,
  6.25,
  4.25,
  8.8125,
  6.40625,
  8.375,
  4.0],
 3.84375]

In [84]:
mean_heatmap

array([0.10833333, 0.02105735, 0.0625    , 0.10611511, 0.0734767 ,
       0.07401033, 0.05617978, 0.05755396, 0.03819095, 0.03888889,
       0.19101124, 0.05395683, 0.22682446, 0.        , 0.06231156,
       0.0242411 , 0.10215054, 0.10453649, 0.04536489, 0.0094086 ,
       0.09346734, 0.13169643, 0.07777778, 0.22767857, 0.0915047 ,
       0.225     , 0.00416667, 0.00414938, 0.13403509, 0.2040201 ,
       0.27482014, 0.        , 0.        , 0.07482014, 0.06797853,
       0.05241935, 0.10663082, 0.01258993, 0.04496403, 0.02242991,
       0.04120603, 0.06919643, 0.39568345, 0.01032702, 0.02329749,
       0.08481262, 0.00280407, 0.09849246, 0.10555556, 0.04676259,
       0.01798561, 0.15654952, 0.26517572, 0.20143885, 0.07194245,
       0.617357  , 0.10267857, 0.        , 0.05527638, 0.15833333,
       0.13169643, 0.        , 0.01798561, 0.0729783 , 0.15779093,
       0.03718638, 0.03472222, 0.26111111, 0.13824561, 0.        ,
       0.10653266, 0.32070175, 0.        , 0.        , 0.2       ,
       0.02284946, 0.04856115, 0.09444444, 0.        , 0.4       ,
       0.08140704, 0.15107914, 0.03888889, 0.        , 0.08243728,
       0.00703518, 0.1522807 , 0.13632653, 0.04723618, 0.        ,
       0.04444444, 0.28801431, 0.375     , 0.06834532, 0.10286225,
       0.36628111, 0.12777778, 0.12096774, 0.01299283, 0.1       ,
       0.15770609, 0.16944444, 0.36094675, 0.        , 0.        ,
       0.03125   , 0.        , 0.17741935, 0.21111111, 0.11157895,
       0.        , 0.        , 0.13258786, 0.02817209, 0.05737705,
       0.47406082, 0.0031362 , 0.00277778, 0.00479233, 0.00138889,
       0.11079137, 0.01478495, 0.16853933, 0.02688172, 0.        ,
       0.07083333, 0.07222222, 0.05591054, 0.        ])

In [71]:
import plotly.express as px
import pandas as pd
import numpy as np

normalized_losses = {}

drop_features = []
drop_ids = [features.index(feature) for feature in drop_features]

features_dropped = [feature for feature in features if feature not in drop_features]

for task_name, losses in negative_task_losses.items():
    base_loss = losses[1]
    losses = [x[0].tolist() for x in losses[0]]
    losses = np.array(losses)
    losses = np.delete(losses, drop_ids)

    losses = losses - base_loss[0]
    
    max_loss = np.max(losses)
    min_loss = np.min(losses)

    losses = (losses - min_loss) / (max_loss - min_loss)
    
    # mean_loss = np.mean(losses - base_loss)

    # base_acc = losses[1][1]

    # accs = [base_acc - loss[1] for loss in losses[0]]
    normalized_losses[task_name] = losses


heatmap = np.zeros((len(task_names), len(features_dropped)))

for i, task_name in enumerate(task_names):
    for j, feature in enumerate(features_dropped):
        heatmap[i, j] = normalized_losses[task_name][j]

heatmap /= np.mean(heatmap, axis=0, keepdims=True)



# heatmap = np.where(heatmap > 0, np.log(heatmap), -10)
# heatmap[np.isnan(heatmap)] = np.min(heatmap[np.isfinite(heatmap)])
# heatmap[np.isinf(heatmap)] = np.max(heatmap[np.isfinite(heatmap)])

# heatmap = np.clip(heatmap, -5, 5)


fig = px.imshow(heatmap, x=[str(x) for x in features_dropped], y=task_names)

fig.show()


NameError: name 'negative_task_losses' is not defined